In [1]:
import os
import csv
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

In [3]:
hmda_vars = ['num_approved_loans', 
             'num_applications',
             'sum_approved_loans',
             'sum_all_loan_amount',
             'denial_rate',
             'avg_approved_loan_size'
             #'median__all_income',
             #'sum_rate_spread',
             #'mean_LTV'
            ]

In [4]:
data = data[data["state"] == 6]

In [5]:
data['pct_minority'] = (data['Tot.Pop'] - data['Tot.WhitePop'])/data['Tot.Pop'] * 100

In [6]:
# finding the median income quintiles
# sort the median__all_income variable in ascending order
data.sort_values(by='pct_minority', inplace=True)

# divide the sorted median__all_income variable into 5 equal parts, or quintiles
data['pct_minority_Quintiles'] = pd.qcut(data['pct_minority'], q=5, labels=False)

# create a new variable called "Median_Income_Quintiles" based on the quintiles
data['pct_minority_Quintiles'] = pd.cut(data['pct_minority'], bins=5, labels=['20', '40', '60', '80', '100'])

In [7]:
Quintiles_loan_table = data.groupby('which_bank')[hmda_vars].mean()
#Quintiles_loan_table['loan_approvel_rate'] = Quintiles_loan_table['num_approved_loans']/Quintiles_loan_table['num_applications']
#Quintiles_loan_table = Quintiles_loan_table.sort_values(by='loan_approvel_rate', ascending=False)
Quintiles_loan_table

,num_approved_loans,num_applications,sum_approved_loans,sum_all_loan_amount,denial_rate,avg_approved_loan_size
which_bank,,,,,,
All Other Banks,51.791350,61.556190,2.700681e+07,3.104069e+07,0.831192,521807.352760
Bank of West,2.465273,3.490267,1.388032e+06,1.871864e+06,0.788260,497403.240472


In [8]:
Quintiles_loan_table3 = data.groupby(['which_bank', "pct_minority_Quintiles"])[hmda_vars].sum()
Quintiles_loan_table3['loan_pct_in_minority_group'] = Quintiles_loan_table3['num_approved_loans']/Quintiles_loan_table3['num_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table3['$loan_pct_in_minority_group'] = Quintiles_loan_table3['sum_approved_loans']/Quintiles_loan_table3['sum_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table3['loan_approval_rate'] = Quintiles_loan_table3['num_approved_loans']/Quintiles_loan_table3['num_applications'].groupby("which_bank").sum()
Quintiles_loan_table3.iloc[:,-3:]

loan_pct_in_minority_group  \
which_bank      pct_minority_Quintiles                               
All Other Banks 20                                        0.241480   
                40                                        0.375884   
                60                                        0.225279   
                80                                        0.125867   
                100                                       0.031490   
Bank of West    20                                        0.271398   
                40                                        0.355917   
                60                                        0.190973   
                80                                        0.142133   
                100                                       0.039579   

                                        $loan_pct_in_minority_group  \
which_bank      pct_minority_Quintiles                                
All Other Banks 20                                         0.275367   
                40                                         0.366164   
                60                                         0.209731   
                80                                         0.116742   
                100                                        0.031995   
Bank of West    20                                         0.317428   
                40                                         0.352759   
                60                                         0.164186   
                80                                         0.125549   
                100                                        0.040077   

                                        loan_approval_rate  
which_bank      pct_minority_Quintiles                      
All Other Banks 20                                0.203173  
                40                                0.316256  
                60                                0.189542  
                80                                0.105900  
                100                               0.026495  
Bank of West    20                                0.191696  
                40                                0.251394  
                60                                0.134889  
                80                                0.100392  
                100                               0.027956

In [9]:
data['pct_White'] = data['Tot.WhitePop']/data['Tot.Pop'] * 100
data['pct_Black'] = data['Tot.BlackPop ']/data['Tot.Pop'] * 100
data['pct_AmericIndian'] = data['Tot.AmericIndianPop']/data['Tot.Pop'] * 100
data['pct_Asian'] = data['Tot.AsianPop']/data['Tot.Pop'] * 100
data['pct_NativeHawaiianPacific'] = data['Tot.NativeHawaiianPacificPop']/data['Tot.Pop'] * 100
data['pct_OtherRaceAlone'] = data['Tot.OtherRaceAlonePop']/data['Tot.Pop'] * 100
data['pct_TwoOrMoreRace'] = data['Tot.TwoOrMoreRace']/data['Tot.Pop'] * 100
data['pct_TwoOrMoreRace(Some other race)'] = data['Tot.TwoOrMoreRace(Some other race)']/data['Tot.Pop'] * 100
data['pct_Hispanic/Latino'] = data['Tot.Hispanic/Latino']/data['Tot.Pop'] * 100
data['pct_NotHispanic/Latino'] = data['Tot.NotHispanic/Latino']/data['Tot.Pop'] * 100

In [10]:
data['major_minority'] = data[['pct_Black', 'pct_AmericIndian', 'pct_Asian','pct_NativeHawaiianPacific',
                                        'pct_OtherRaceAlone','pct_TwoOrMoreRace','pct_TwoOrMoreRace(Some other race)',
                                        'pct_Hispanic/Latino']].idxmax(axis=1)
data['major_minority'] = data['major_minority'].apply(lambda x: 'Black' if x=='pct_Black'
                                                      else 'American Indian' if x=='pct_AmericIndian' 
                                                      else 'Asian' if x=='pct_Asian' 
                                                      else 'Native Hawaiian Pacific' if x=='pct_NativeHawaiianPacific' 
                                                      else 'Other Race Alone' if x=='pct_OtherRaceAlone' 
                                                      else 'Two Or More Race' if x=='pct_TwoOrMoreRace' 
                                                      else 'Two Or More Race(Some other race)' if x=='pct_TwoOrMoreRace(Some other race)' 
                                                      else 'Hispanic/Latino')

In [11]:
Quintiles_loan_table4 = data.groupby(['which_bank', "major_minority"])[hmda_vars].sum()
Quintiles_loan_table4['loan_pct_in_minority_group'] = Quintiles_loan_table4['num_approved_loans']/Quintiles_loan_table4['num_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table4['$loan_pct_in_minority_group'] = Quintiles_loan_table4['sum_approved_loans']/Quintiles_loan_table4['sum_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table4['loan_approval_rate'] = Quintiles_loan_table4['num_approved_loans']/Quintiles_loan_table4['num_applications'].groupby("which_bank").sum()
Quintiles_loan_table4.iloc[:,-3:]

loan_pct_in_minority_group  \
which_bank      major_minority                                        
All Other Banks American Indian                            0.001217   
                Asian                                      0.279453   
                Black                                      0.016930   
                Hispanic/Latino                            0.691110   
                Native Hawaiian Pacific                    0.000022   
                Other Race Alone                           0.000284   
                Two Or More Race                           0.010984   
Bank of West    American Indian                            0.000877   
                Asian                                      0.380484   
                Black                                      0.017840   
                Hispanic/Latino                            0.580133   
                Other Race Alone                           0.001170   
                Two Or More Race                           0.019497   

                                         $loan_pct_in_minority_group  \
which_bank      major_minority                                         
All Other Banks American Indian                             0.000753   
                Asian                                       0.365491   
                Black                                       0.016531   
                Hispanic/Latino                             0.594778   
                Native Hawaiian Pacific                     0.000008   
                Other Race Alone                            0.000487   
                Two Or More Race                            0.021952   
Bank of West    American Indian                             0.000983   
                Asian                                       0.486679   
                Black                                       0.015251   
                Hispanic/Latino                             0.461989   
                Other Race Alone                            0.001106   
                Two Or More Race                            0.033991   

                                         loan_approval_rate  
which_bank      major_minority                               
All Other Banks American Indian                    0.001024  
                Asian                              0.235122  
                Black                              0.014245  
                Hispanic/Latino                    0.581477  
                Native Hawaiian Pacific            0.000019  
                Other Race Alone                   0.000239  
                Two Or More Race                   0.009242  
Bank of West    American Indian                    0.000620  
                Asian                              0.268746  
                Black                              0.012601  
                Hispanic/Latino                    0.409764  
                Other Race Alone                   0.000826  
                Two Or More Race                   0.013771

In [12]:
data['white_majority'] = (data['pct_White'] > data['pct_minority'])

In [13]:
loan_table5 = data.groupby(['which_bank', "white_majority"])[hmda_vars].sum()
loan_table5['loan_pct'] = loan_table5['num_approved_loans']/loan_table5['num_approved_loans'].groupby("which_bank").sum()
loan_table5['$loan_pct'] = loan_table5['sum_approved_loans']/loan_table5['sum_approved_loans'].groupby("which_bank").sum()
loan_table5['loan_approval_rate'] = loan_table5['num_approved_loans']/loan_table5['num_applications'].groupby("which_bank").sum()
loan_table5.iloc[:,-3:]

loan_pct  $loan_pct  loan_approval_rate
which_bank      white_majority                                         
All Other Banks False           0.254556   0.238932            0.214175
                True            0.745444   0.761068            0.627192
Bank of West    False           0.267596   0.237623            0.189011
                True            0.732404   0.762377            0.517317